In [1]:
import pandas as pd

from pandas.io.json import json_normalize
from SPARQLWrapper import SPARQLWrapper, JSON

In [2]:
def query(s):
    # this will run the query and turn the results into a table that you can process
    # results are returned as a pandas dataframe
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    
    sparql.setQuery(s)
    sparql.setReturnFormat(JSON)

    res = sparql.query().convert()
    return json_normalize(res["results"]["bindings"])

## Run our SPARQL query

In [3]:
s = """SELECT ?drug ?drugLabel ?gene ?geneLabel ?entrez_id ?disease ?diseaseLabel WHERE {
      ?drug wdt:P129 ?gene_product .   # drug interacts with a gene_product 
      ?gene_product wdt:P702 ?gene .  # gene_product is encoded by a gene
      ?gene wdt:P2293 ?disease .    # gene is genetically associated with a disease 
      ?gene wdt:P351 ?entrez_id .  # get the entrez gene id for the gene 
      # add labels
      SERVICE wikibase:label {
            bd:serviceParam wikibase:language "en" .
      }
    }
    limit %d
    """ % (1000)

res = query(s)

In [4]:
res.shape

(595, 17)

In [5]:
res.head()

,disease.type,disease.value,diseaseLabel.type,diseaseLabel.value,diseaseLabel.xml:lang,drug.type,drug.value,drugLabel.type,drugLabel.value,drugLabel.xml:lang,entrez_id.type,entrez_id.value,gene.type,gene.value,geneLabel.type,geneLabel.value,geneLabel.xml:lang
0,uri,http://www.wikidata.org/entity/Q12174,literal,obesity,en,uri,http://www.wikidata.org/entity/Q60235,literal,Caffeine,en,literal,140,uri,http://www.wikidata.org/entity/Q4682275,literal,adenosine A3 receptor,en
1,uri,http://www.wikidata.org/entity/Q12174,literal,obesity,en,uri,http://www.wikidata.org/entity/Q190012,literal,Adenosine,en,literal,140,uri,http://www.wikidata.org/entity/Q4682275,literal,adenosine A3 receptor,en
2,uri,http://www.wikidata.org/entity/Q12174,literal,obesity,en,uri,http://www.wikidata.org/entity/Q407308,literal,Theophylline,en,literal,140,uri,http://www.wikidata.org/entity/Q4682275,literal,adenosine A3 receptor,en
3,uri,http://www.wikidata.org/entity/Q12174,literal,obesity,en,uri,http://www.wikidata.org/entity/Q729213,literal,Nicardipine,en,literal,140,uri,http://www.wikidata.org/entity/Q4682275,literal,adenosine A3 receptor,en
4,uri,http://www.wikidata.org/entity/Q12174,literal,obesity,en,uri,http://www.wikidata.org/entity/Q905783,literal,Istradefylline,en,literal,140,uri,http://www.wikidata.org/entity/Q4682275,literal,adenosine A3 receptor,en


Our results from the SPARQL query has more detail than we care about. We only want to know the names of the drug, disease, and gene columns.

## Extract only the columns we care about

In [6]:
good = (res
    [["drugLabel.value", "diseaseLabel.value", "geneLabel.value"]]
    .rename(columns = lambda col: col.replace("Label.value", ""))
)

In [7]:
good.head()

,drug,disease,gene
0,Caffeine,obesity,adenosine A3 receptor
1,Adenosine,obesity,adenosine A3 receptor
2,Theophylline,obesity,adenosine A3 receptor
3,Nicardipine,obesity,adenosine A3 receptor
4,Istradefylline,obesity,adenosine A3 receptor


We have just grabbed the three columns we care about and renamed them.

## Count the number of genes per (drug, disease) pair

How many genes link each unique (drug, disease) pair? After counting, order the (drug, disease) pairs in descending order of number of linking genes.

In [8]:
counts = (good
    .groupby(["drug", "disease"], as_index = False) # group into unique (drug, disease) pairs
    .size() # count the number of genes linking this (drug, disease) pair
    .rename("gene_count") # name the result "gene_count"
    .to_frame() # change to a pandas dataframe from a series
    .reset_index() # make the drug and disease individual columns
    .sort_values("gene_count", ascending = False) # sort in descending order
    .reset_index(drop = True) # order rows from 0 onwards
)

In [9]:
counts.head()

,drug,disease,gene_count
0,Caffeine,obesity,3
1,Quisinostat,obesity,2
2,Adenosine triphosphate,obesity,2
3,Trichostatin A,obesity,2
4,Panobinostat,obesity,2


## Add the genes used to link each (drug, disease) pair

Now add another column containing the actual genes linking each (drug, disease) pair.

In [10]:
counts["genes"] = counts[["drug", "disease"]].apply(
    lambda row:
        list(good.query('drug == "{}" & disease == "{}"'.format(row["drug"], row["disease"]))["gene"]),
    axis = 1
)

In [11]:
counts.head()

,drug,disease,gene_count,genes
0,Caffeine,obesity,3,"[adenosine A3 receptor, inositol 1,4,5-trispho..."
1,Quisinostat,obesity,2,"[histone deacetylase 7, histone deacetylase 9]"
2,Adenosine triphosphate,obesity,2,"[inositol 1,4,5-trisphosphate receptor, type 1..."
3,Trichostatin A,obesity,2,"[histone deacetylase 7, histone deacetylase 9]"
4,Panobinostat,obesity,2,"[histone deacetylase 7, histone deacetylase 9]"


## Save to file

In [12]:
counts.to_csv("drug_disease_count_toby.tsv", sep = '\t', index = False)

---

## Make cytoscape file

We want a file with all the edges for Cytoscape.

In [13]:
drug_gene = (good
    [["drug", "gene"]] # only look at the drug and gene columns
    .assign( # create new columns with values for the source, edge, and target types
        source_type = "drug",
        edge_type = "interacts_with",
        target_type = "gene"
    )
    .rename(columns = {"drug": "source_node", "gene": "target_node"}) # rename drug and gene columns
)

In [14]:
drug_gene.head()

,source_node,target_node,edge_type,source_type,target_type
0,Caffeine,adenosine A3 receptor,interacts_with,drug,gene
1,Adenosine,adenosine A3 receptor,interacts_with,drug,gene
2,Theophylline,adenosine A3 receptor,interacts_with,drug,gene
3,Nicardipine,adenosine A3 receptor,interacts_with,drug,gene
4,Istradefylline,adenosine A3 receptor,interacts_with,drug,gene


In [15]:
# do the same for disease gene edges
disease_gene = (good
    [["disease", "gene"]]
    .assign(
        source_type = "disease",
        edge_type = "associated_with",
        target_type = "gene"
    )
    .rename(columns = {"disease": "source_node", "gene": "target_node"})
)

In [16]:
disease_gene.head()

,source_node,target_node,edge_type,source_type,target_type
0,obesity,adenosine A3 receptor,associated_with,disease,gene
1,obesity,adenosine A3 receptor,associated_with,disease,gene
2,obesity,adenosine A3 receptor,associated_with,disease,gene
3,obesity,adenosine A3 receptor,associated_with,disease,gene
4,obesity,adenosine A3 receptor,associated_with,disease,gene


## Group the two edge types together

In [17]:
final = (pd
    .concat([drug_gene, disease_gene])
    [["source_node", "source_type", "edge_type", "target_node", "target_type"]]
)

In [18]:
final.head()

,source_node,source_type,edge_type,target_node,target_type
0,Caffeine,drug,interacts_with,adenosine A3 receptor,gene
1,Adenosine,drug,interacts_with,adenosine A3 receptor,gene
2,Theophylline,drug,interacts_with,adenosine A3 receptor,gene
3,Nicardipine,drug,interacts_with,adenosine A3 receptor,gene
4,Istradefylline,drug,interacts_with,adenosine A3 receptor,gene


## Save file to disk

In [19]:
final.to_csv("drug_gene_disease_network_toby.txt", sep = '\t', index = False)